In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('frame_20.txt', sep=' ', header=None)

In [3]:
df

,0,1,2,3,4
0,0,0.878906,0.422917,0.242188,0.576389
1,1,0.550781,0.404861,0.335938,0.548611
2,2,0.117578,0.500000,0.235156,1.000000
3,0,0.882422,0.399306,0.235156,0.531944
4,1,0.552344,0.388194,0.331250,0.501389
5,2,0.116797,0.498611,0.230469,0.994444


In [4]:
df.columns = ["id", "x", "y", "width", "height"]

In [6]:
frameW = 1280
frameH = 720

In [9]:
ids = df.loc[df['id'] == 0]

In [25]:
def IoU(bb1, bb2, frameW, frameH):
    x1 = bb1['x'] * frameW
    y1 = bb1['y'] * frameH
    x2 = bb2['x'] * frameW
    y2 = bb2['y'] * frameH
    w1 = bb1['width'] * frameW
    h1 = bb1['height'] * frameH
    w2 = bb2['width'] * frameW
    h2 = bb2['height'] * frameH
    total_area = w1 * h1 + w2 * h2
    # determine the (x, y)-coordinates of the intersection rectangle
    xA = max(x1, x2)
    yA = max(y1, y2)
    xB = min(x1 + w1, x2 + w2)
    yB = min(y1 + h1, y2 + h2)
    # compute the area of intersection rectangle
    interArea = max(0, xB - xA) * max(0, yB - yA)
    # compute the total area covered
    boxArea = total_area - interArea
    # compute the intersection over union
    iou = interArea / float(boxArea)
    return iou

In [45]:
bb1 = ids.iloc[0].drop('id')
bb1

x         0.140625
y         0.500000
width     0.281250
height    1.000000
Name: 2, dtype: float64

In [15]:
bb2 = ids.iloc[1].drop('id')
bb2

In [46]:
IoU(bb1, bb2, 1280, 720)

0.8685870212240058

In [23]:
df['id'].max()

2

In [28]:
for i in range(df['id'].max() + 1):
    ids = df.loc[df['id'] == i]
    bb1 = ids.iloc[0].drop('id')
    bb2 = ids.iloc[1].drop('id')
    print(IoU(bb1, bb2, frameW, frameH))

0.8235633286895309
0.8458748890734938
0.9654653039706026


In [31]:
import glob

In [47]:
files = glob.glob("vid12/*.txt")
files

['vid12\\frame_000.txt',
 'vid12\\frame_100.txt',
 'vid12\\frame_200.txt',
 'vid12\\frame_300.txt',
 'vid12\\frame_400.txt',
 'vid12\\frame_500.txt',
 'vid12\\frame_600.txt',
 'vid12\\frame_700.txt',
 'vid12\\frame_800.txt']

In [34]:
iou_list = []
fp = 0
fn = 0

In [35]:
for file in files:
    df = pd.read_csv(file, sep=' ', header=None)
    df.columns = ["id", "x", "y", "width", "height"]

    for i in range(df['id'].max() + 1):
        ids = df.loc[df['id'] == i]
        if len(ids) == 2:
            bb1 = ids.iloc[0].drop('id')
            bb2 = ids.iloc[1].drop('id')
            iou_list.append(IoU(bb1, bb2, frameW, frameH))
        else:
            fp += 1

In [37]:
import numpy as np

In [38]:
np.mean(iou_list)

0.8147501200102073

In [39]:
df

,id,x,y,width,height
0,0,0.585938,0.475000,0.310937,0.508333
1,1,0.886719,0.467361,0.226562,0.534722
2,2,0.140625,0.500000,0.281250,1.000000
3,0,0.587500,0.455556,0.309375,0.469444
4,1,0.892578,0.443056,0.214844,0.486111
5,2,0.150781,0.500694,0.301562,0.998611


In [44]:
df.index[df['id'] == 0].tolist()[1]

3